1. 請利用 Keras 內建的 ResNet50 預訓練模型並利用轉移學習(Transfer Learning)的方式來完成貓狗辨識(貓狗資料集可以使用 Kaggle 的貓狗資料集)。

資料庫：[Kaggle data set cats & dogs](https://www.kaggle.com/datasets/tongpython/cat-and-dog?resource=download)

程式設計：

In [11]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input , decode_predictions
from tensorflow.keras.utils import to_categorical
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image
import os

# 載入資料集
trainC_path = './CandD/training_set/cats'
testC_path = './CandD/test_set/cats'
trainD_path = './CandD/training_set/dogs'
testD_path = './CandD/test_set/dogs'

# 載入一張圖做為測試

image_path = f'{trainC_path}/cat.1.jpg'
img = image.load_img(image_path, target_size = (224, 224))
# plt.imshow(img)
x = image.img_to_array(img)
print(x.shape, "ori")
x = x.astype(np.uint8)
x = np.expand_dims(x, axis = 0)
print(x.shape, "added")
print(x.dtype)

(224, 224, 3) ori
(1, 224, 224, 3) added
uint8


In [12]:
# load and preprocess image into ndarray
def load(dataset_dir):
    images = []
    for filename in os.listdir(dataset_dir):
        if filename.endswith(".jpg") or filename.endswith(".png"):  # Adjust the file extensions based on your dataset
            file_path = os.path.join(dataset_dir, filename)
            img = image.load_img(file_path, target_size = (224, 224))

            # Convert the image to a numpy array
            image_array = np.array(img)
            # Convert float into uint8
            image_array = image_array.astype(np.uint8)
            # Append the image array to the list
            images.append(image_array)
            
    compiled_images = np.stack(images, axis=0)
    return compiled_images

trainC = load(trainC_path)
testC = load(testC_path)
trainD = load(trainD_path)
testD = load(testD_path)

In [13]:
# Create Label
# 0 = cat
# 1 = dog

lentc = len(trainC)
lenttc = len(testC)
lentd = len(trainD)
lenttd = len(testD)
cat = 0
dog = 1
def createlab(num, label):
    labels = np.full((num,), label)
    return labels

trainC_label = createlab(lentc, cat)
testC_label = createlab(lenttc, cat)
trainD_label = createlab(lentd, dog)
testD_label = createlab(lenttd, dog)


In [15]:
# 分別串接貓狗 訓練及測試集

train_data = np.concatenate((trainC, trainD), axis=0)
train_label = np.concatenate((trainC_label, trainD_label), axis=0)
test_data = np.concatenate((testC, testD), axis=0)
test_label = np.concatenate((testC_label, testD_label), axis=0)


# 指定亂數種子
seed = 10
# 替下一個 random function 設定亂數種子
np.random.seed(seed)
# 打亂 2 個 Numpy 陣列
def randomize(a, b):
    permutation = np.random.permutation(b.shape[0])
    shuffled_a = a[permutation,:,:]
    shuffled_b = b[permutation]
    return shuffled_a, shuffled_b
    
# 將資料集打散
train_data, train_label = randomize(train_data, train_label)

# one_hot encoding
train_label = to_categorical(train_label,2)
test_label= to_categorical(test_label,2)

In [16]:
# 載入預先訓練好的模型 -- ResNet50
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
print(resnet_model.summary())

# 訓練資料的資料前處理
# 將訓練資料縮放到 [-1,1]的區間
train_Data_new = train_data.astype("float32")
preprocess_train_input = preprocess_input(train_Data_new)
print(train_Data_new.shape) 
# 測試資料的資料前處理
test_Data_new = test_data.astype("float32")
preprocess_test_input = preprocess_input(test_Data_new)
print(test_Data_new.shape) 

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [48]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D

for layer in resnet_model.layers:
    layer.trainable = False

# 模型架構 3rd
model = Sequential()
model.add(resnet_model)
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.5))
model.add(Dense(2, activation="softmax"))

In [49]:
# 編譯模型3rd
model.compile(loss="categorical_crossentropy", optimizer="adam",
              metrics=["accuracy"])
# 訓練模型3rd
history = model.fit(train_Data_new, train_label,
                    validation_data=(test_Data_new, test_label), 
                    epochs=14, batch_size=32, verbose=2)

Epoch 1/14
251/251 - 60s - loss: 0.6679 - accuracy: 0.5455 - val_loss: 0.0798 - val_accuracy: 0.9733 - 60s/epoch - 239ms/step
Epoch 2/14
251/251 - 53s - loss: 0.6411 - accuracy: 0.5507 - val_loss: 0.0782 - val_accuracy: 0.9743 - 53s/epoch - 213ms/step
Epoch 3/14
251/251 - 54s - loss: 0.6363 - accuracy: 0.5488 - val_loss: 0.0671 - val_accuracy: 0.9792 - 54s/epoch - 214ms/step
Epoch 4/14
251/251 - 53s - loss: 0.6338 - accuracy: 0.5544 - val_loss: 0.1037 - val_accuracy: 0.9669 - 53s/epoch - 213ms/step
Epoch 5/14
251/251 - 53s - loss: 0.6358 - accuracy: 0.5503 - val_loss: 0.0886 - val_accuracy: 0.9708 - 53s/epoch - 212ms/step
Epoch 6/14
251/251 - 53s - loss: 0.6397 - accuracy: 0.5518 - val_loss: 0.0799 - val_accuracy: 0.9768 - 53s/epoch - 212ms/step
Epoch 7/14
251/251 - 53s - loss: 0.6323 - accuracy: 0.5425 - val_loss: 0.0773 - val_accuracy: 0.9787 - 53s/epoch - 212ms/step
Epoch 8/14
251/251 - 53s - loss: 0.6309 - accuracy: 0.5477 - val_loss: 0.1581 - val_accuracy: 0.9565 - 53s/epoch - 213

In [51]:
loss, accuracy = model.evaluate(test_Data_new, test_label, verbose=0)
print("測試資料集的準確度 = {:.2f}".format(accuracy))

測試資料集的準確度 = 0.98


In [52]:
model.save('cat_dog3.h5')

In [53]:
from tensorflow.keras.models import load_model

model = load_model('cat_dog3.h5')

In [54]:
from tensorflow.keras.preprocessing import image

# 測試網路上找的圖片
# Load and preprocess the cat image
cat_image = image.load_img('cat.jpg', target_size=(224, 224))
cat_image = image.img_to_array(cat_image)
cat_image = preprocess_input(cat_image)
cat_image = np.expand_dims(cat_image, axis=0)
print(cat_image.shape)
predictions = model.predict(cat_image)
class_index = np.argmax(predictions[0])

if class_index == 0:
    print("The image contains a cat.")
else:
    print("The image does not contain a cat.")

(1, 224, 224, 3)
1/1 [==============================] - 2s 2s/step
The image contains a cat.


In [55]:
print(f"Tensorflow Version: {tf.__version__}")
print(f"Number of GPUs Available: {len(tf.config.list_physical_devices('GPU'))}")

Tensorflow Version: 2.12.0
Number of GPUs Available: 1


In [58]:
import visualkeras
visualkeras.layered_view(model)
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d_3   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dropout_4 (Dropout)         (None, 2048)              0         
                                                                 
 dense_5 (Dense)             (None, 2)                 4098      
                                                                 
Total params: 23,591,810
Trainable params: 4,098
Non-trainable params: 23,587,712
_________________________________________________________________
None
